Hey ofac peeps. Here's an example of loading json data into a database (in this case sqlite) via python.

A lot of the ofac data is stored as a nested dictionary, so you'll need to do some stuff iterating through the keys to get what you want/clean the values up, but once you do that uploading it into a database should be very trivial.

Below I explore some data and show a general process of how to insert the data once you've cleaned some of the json up and stuff.


In [1]:
import dataset
from sqlalchemy.exc import ProgrammingError
import json
import sqlite3

In [ ]:
import sys
print sys.

In [138]:
# # Set up data base stuff
DB_STRING = "sqlite:///ofac.db"
TABLE_NAME = "ofac"
# Set up data base
db = dataset.connect(DB_STRING)

In [3]:
conn = sqlite3.connect('ofac.db')

In [8]:
# the json package has two main methods
# .load() converts json to python (dict or sometimes a list)
# .dump() will 'dump' a dict into json

# we'll use load to load in the json


traffic = json.load(open("OFAC.JSON"))

# If you run `traffic` by itself to view the content it may crash your kernel, heads up 

# traffic

{u'sdnList': {u'_xmlns': u'http://tempuri.org/sdnList.xsd',
  u'_xmlns:xsi': u'http://www.w3.org/2001/XMLSchema-instance',
  u'publshInformation': {u'Publish_Date': u'02/16/2017',
   u'Record_Count': u'5533'},
  u'sdnEntry': [{u'addressList': {u'address': {u'city': u'Havana',
      u'country': u'Cuba',
      u'uid': u'25'}},
    u'akaList': {u'aka': {u'category': u'strong',
      u'lastName': u'AERO-CARIBBEAN',
      u'type': u'a.k.a.',
      u'uid': u'12'}},
    u'lastName': u'AEROCARIBBEAN AIRLINES',
    u'programList': {u'program': u'CUBA'},
    u'sdnType': u'Entity',
    u'uid': u'36'},
   {u'addressList': {u'address': {u'address1': u'Ibex House, The Minories',
      u'city': u'London',
      u'country': u'United Kingdom',
      u'postalCode': u'EC3N 1DY',
      u'uid': u'129'}},
    u'akaList': {u'aka': {u'category': u'strong',
      u'lastName': u'AVIA IMPORT',
      u'type': u'a.k.a.',
      u'uid': u'57'}},
    u'lastName': u'ANGLO-CARIBBEAN CO., LTD.',
    u'programList': {u'p

In [79]:
# This is just exploring the json

print type(traffic)
print traffic.keys()
print traffic['sdnList'].keys()
print type(traffic['sdnList']['sdnEntry'])
print len(traffic['sdnList']['sdnEntry'])
print traffic['sdnList']['sdnEntry'][0].keys()
traffic['sdnList']['sdnEntry'][10]

<type 'dict'>
[u'sdnList']
[u'_xmlns:xsi', u'publshInformation', u'_xmlns', u'sdnEntry']
<type 'list'>
5533
[u'uid', u'akaList', u'lastName', u'addressList', u'sdnType', u'programList']


{u'addressList': {u'address': {u'country': u'Panama', u'uid': u'328'}},
 u'lastName': u'COMERCIAL CIMEX, S.A.',
 u'programList': {u'program': u'CUBA'},
 u'sdnType': u'Entity',
 u'uid': u'551'}

In [54]:
# Problem: Looks like you guys have different keys for different observations - so data will 
#    probably have some NA values - which is fine

# Here are the keys for the first 10 values
data = traffic['sdnList']['sdnEntry'][0:10]
for i in data:
    print i.keys()

[u'uid', u'akaList', u'lastName', u'addressList', u'sdnType', u'programList']
[u'uid', u'akaList', u'lastName', u'addressList', u'sdnType', u'programList']
[u'uid', u'akaList', u'lastName', u'addressList', u'sdnType', u'programList']
[u'lastName', u'addressList', u'uid', u'sdnType', u'programList']
[u'lastName', u'addressList', u'uid', u'sdnType', u'programList']
[u'lastName', u'addressList', u'uid', u'sdnType', u'programList']
[u'lastName', u'addressList', u'uid', u'sdnType', u'programList']
[u'lastName', u'addressList', u'uid', u'sdnType', u'programList']
[u'lastName', u'addressList', u'uid', u'sdnType', u'programList']
[u'uid', u'akaList', u'lastName', u'addressList', u'sdnType', u'programList']


In [123]:
# However, look at this value

traffic['sdnList']['sdnEntry'][2]['addressList']

{u'address': [{u'address1': u'Zweierstrasse 35',
   u'city': u'Zurich',
   u'country': u'Switzerland',
   u'postalCode': u'CH-8022',
   u'uid': u'199'},
  {u'address1': u'Avenida de Concha Espina 8',
   u'city': u'Madrid',
   u'country': u'Spain',
   u'postalCode': u'E-28036',
   u'uid': u'200'},
  {u'address1': u'Dai-Ichi Bldg. 6th Floor, 10-2 Nihombashi, 2-chome, Chuo-ku',
   u'city': u'Tokyo',
   u'country': u'Japan',
   u'postalCode': u'103',
   u'uid': u'201'},
  {u'address1': u'Federico Boyd Avenue & 51 Street',
   u'city': u'Panama City',
   u'country': u'Panama',
   u'uid': u'202'}]}

The address key has multiple entries, including one of which that's a list. Iterating through this will produce

`TypeError: list indices must be integers, not str`

So that's something you'll need to take into account - non-uniform values


Once you sort through that stuff, you can just use the following code structure.

Just add column names as you get them - e.g. if you want to add a column called `ofac`, do:

   `try:`
      `table.insert(dict(`
      `     ofac = i['ofac']`
      `     ))`

In [149]:
# stream through each element in the list
# for each element in list:
#    insert list's elements into db
table = db[TABLE_NAME]
data = traffic['sdnList']['sdnEntry'][0:7]

for i in data:
            
        try:
            table.insert(dict(
                uid      = i['uid'],
                lastname = i['lastName'],
                sdnType  = i['sdnType']
            ))
            
        except ProgrammingError as err:
            print(err)

In [147]:
# Verify that it worked
conn = sqlite3.connect("ofac.db")
query = "SELECT * FROM ofac"
conn.execute(query).fetchall()

[(1, None, u'306', u'Entity', u'BANCO NACIONAL DE CUBA'),
 (2, None, u'475', u'Entity', u'CASA DE CUBA'),
 (3, None, u'306', u'Entity', u'BANCO NACIONAL DE CUBA'),
 (4, None, u'475', u'Entity', u'CASA DE CUBA'),
 (5, None, u'36', u'Entity', u'AEROCARIBBEAN AIRLINES'),
 (6, None, u'173', u'Entity', u'ANGLO-CARIBBEAN CO., LTD.'),
 (7, None, u'306', u'Entity', u'BANCO NACIONAL DE CUBA'),
 (8, None, u'424', u'Entity', u'BOUTIQUE LA MAISON'),
 (9, None, u'475', u'Entity', u'CASA DE CUBA'),
 (10, None, u'480', u'Entity', u'CECOEX, S.A.'),
 (11, None, u'535', u'Entity', u'CIMEX')]

### For readability or POC. - can read it into Python with pandas

In [148]:
import pandas as pd

# we'll read in directly from the query
data_in_pandas = pd.read_sql_query(query, conn)
data_in_pandas

,id,city,uid,sdnType,lastname
0,1,None,306,Entity,BANCO NACIONAL DE CUBA
1,2,None,475,Entity,CASA DE CUBA
2,3,None,306,Entity,BANCO NACIONAL DE CUBA
3,4,None,475,Entity,CASA DE CUBA
4,5,None,36,Entity,AEROCARIBBEAN AIRLINES
5,6,None,173,Entity,"ANGLO-CARIBBEAN CO., LTD."
6,7,None,306,Entity,BANCO NACIONAL DE CUBA
7,8,None,424,Entity,BOUTIQUE LA MAISON
8,9,None,475,Entity,CASA DE CUBA
9,10,None,480,Entity,"CECOEX, S.A."


# tl;dr: The json is pretty messy but once you clean it up uploading it into sqlite should be easy